# [1단계] 환경 설정 및 라이브러리 설치

## 1-1. 필수 라이브러리 설치

In [10]:
!pip install -q -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121  # CUDA 12.1 빌드 [5]
!pip install -q -U transformers datasets accelerate peft trl==0.12.2 huggingface_hub               # TRL 0.12.2 명시 [13][15]
!pip install -q flash-attn --no-build-isolation                                                     # FlashAttention 설치 [7]
!pip install -q liger-kernel                                                                         # Liger-Kernel 안정판 [5]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 6.4 MB/s eta 0:00:00


## 1-2. 필요한 라이브러리 임포트

In [11]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from huggingface_hub import login
from liger_kernel.transformers import apply_liger_kernel_to_qwen2 # Liger Kernel 임포트

print("✅ 라이브러리 설치 및 임포트 완료!")

✅ 라이브러리 설치 및 임포트 완료!


## 1-3. Hugging Face 로그인

In [12]:
# RunPod 터미널에서 'export HF_TOKEN=hf_...' 명령어로 토큰을 미리 설정했다고 가정.
hf_token = os.environ.get('HF_TOKEN')

# Colab/Jupyter 테스트를 위한 임시 코드
if hf_token is None:
    print("RunPod 환경 변수를 찾을 수 없어 Colab userdata로 대체합니다.")
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')

# 로그인 실행
if hf_token:
    login(token=hf_token)
    print("✅ Hugging Face 로그인 성공!")
else:
    print("🚨 Hugging Face 토큰을 찾을 수 없습니다.")

RunPod 환경 변수를 찾을 수 없어 Colab userdata로 대체합니다.
✅ Hugging Face 로그인 성공!


# [2단계] Instruct 모델 및 토크나이저 로드

## 2-1. 베이스 모델 ID 정의

In [26]:
# 파인튜닝할 베이스 모델을 Instruct 버전으로 직접 지정한다.
model_id = "Qwen/Qwen2.5-14B-Instruct"

## 2-2. Instruct 모델 로드

In [27]:
# 양자화 없이 bfloat16으로, Flash Attention 2를 적용하여 로드한다.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2" # 멘토님 조언 1
)
print(f"--- Instruct 모델 '{model_id}' 로딩 완료 ---")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.70G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

--- Instruct 모델 'Qwen/Qwen2.5-14B-Instruct' 로딩 완료 ---


## 2-3. Liger-Kernel 적용 (멘토님 조언 6)

In [28]:
# 로드된 모델 객체에 Liger Kernel을 적용하여 추가적인 속도 향상을 꾀한다.
try:
    from liger_kernel.transformers import apply_liger_kernel_to_qwen2
    apply_liger_kernel_to_qwen2()
    print("--- Liger-Kernel 적용 완료 ---")
except ImportError:
    print("❗️ Liger-Kernel 라이브러리를 찾을 수 없어 적용을 건너뜁니다.")

Applied Liger kernels to Qwen2
--- Liger-Kernel 적용 완료 ---


## 2-4. 토크나이저 로드

In [29]:
# Instruct 모델에 맞는 토크나이저를 불러온다.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 패딩 토큰 설정
# Instruct 모델의 토크나이저에는 이미 필요한 모든 설정이 되어있지만,
# pad_token이 없는 경우를 대비해 eos_token으로 설정하는 코드는 유지하는 것이 안전하다.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("--- 토크나이저 로딩 완료 ---")
print(f"Pad token: {tokenizer.pad_token}")

--- 토크나이저 로딩 완료 ---
Pad token: <|endoftext|>


# [3단계] 최종 파인튜닝 데이터셋 로드

## 3-1. 데이터셋 ID 정의 및 로드

In [31]:
# RunPod 환경 변수에서 허깅페이스 사용자 이름을 가져온다.
hf_username = os.environ.get('HF_USERNAME')

# Colab/Jupyter 테스트를 위한 임시 코드
if hf_username is None:
    print("RunPod 환경 변수를 찾을 수 없어 Colab userdata로 대체합니다.")
    hf_username = userdata.get('HF_USERNAME')

# 데이터셋의 전체 주소를 완성한다.
dataset_repo_id = f"{hf_username}/combined-dataset-30K-final"

# 허깅페이스 Hub에서 데이터셋을 로드한다.
# DatasetDict 형태로 'train'과 'test' 스플릿이 모두 로드된다.
final_dataset = load_dataset(dataset_repo_id)

print(f"--- 최종 데이터셋 '{dataset_repo_id}' 로드 완료 ---")
print(final_dataset)

RunPod 환경 변수를 찾을 수 없어 Colab userdata로 대체합니다.
--- 최종 데이터셋 'rucipheryn/combined-dataset-30K-final' 로드 완료 ---
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 30606
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1708
    })
})


# [4단계] LoRA 설정

## 4-1. LoRA Config 생성

In [32]:
# peft 라이브러리를 사용해 LoRA 설정을 정의한다.
lora_config = LoraConfig(
    r=16,  # LoRA rank. 높을수록 파라미터 수가 많아지고 표현력이 좋아지지만, 메모리를 더 차지. 16은 표준적인 값.
    lora_alpha=32,  # LoRA 스케일링. 보통 r의 2배수로 설정.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Qwen2 모델의 어텐션 및 MLP 레이어에 LoRA를 적용.
    lora_dropout=0.05,  # 훈련 중 5%의 뉴런을 랜덤하게 비활성화하여 과적합 방지.
    bias="none",  # bias 가중치는 훈련하지 않음.
    task_type="CAUSAL_LM",  # 작업 유형을 '인과 관계 언어 모델링'으로 명시.
)

## 4-2. 모델에 LoRA 적용

In [33]:
# get_peft_model 함수를 사용해 원본 모델에 LoRA 설정을 적용(wrapping)한다.
# 이제부터 이 'lora_model'을 훈련시키면, 원본 모델의 가중치는 얼어붙고 LoRA 레이어만 학습된다.
lora_model = get_peft_model(model, lora_config)

print("--- 모델에 LoRA 적용 완료 ---")

# 훈련 가능한 파라미터 수와 비율을 출력해서 LoRA가 잘 적용되었는지 확인한다.
lora_model.print_trainable_parameters()

--- 모델에 LoRA 적용 완료 ---
trainable params: 68,812,800 || all params: 14,838,846,464 || trainable%: 0.4637


# [5단계] 훈련 계획 수립 (SFTConfig 사용) (멘토님 조언 5)

In [34]:
# [5단계] 훈련 계획 수립 (SFTConfig, Colab/RunPod 호환)

from trl import SFTConfig

## 5-1. wandb 연동 설정 (환경에 따라 자동 전환)
# 먼저 RunPod의 환경 변수를 확인한다.
wandb_api_key = os.environ.get('WANDB_API_KEY')
report_to = "none" # 기본값은 비활성화

# RunPod 환경 변수가 없는 경우, Colab 환경인지 확인한다.
if wandb_api_key is None:
    try:
        from google.colab import userdata
        wandb_api_key = userdata.get('WANDB_API_KEY')
        print("RunPod 환경 변수를 찾을 수 없어 Colab userdata로 wandb 키를 확인합니다.")
    except ImportError:
        # Colab이 아닌 환경에서는 userdata가 없으므로 pass.
        pass

# 최종적으로 API 키 존재 여부에 따라 report_to 설정
if wandb_api_key:
    report_to = "wandb"
    # wandb에 프로젝트 이름을 명시적으로 설정 (선택 사항이지만 추천)
    os.environ['WANDB_PROJECT'] = "sk-networks-ai-camp-final"
    print("✅ wandb 연동이 활성화되었습니다.")
else:
    print("❗️ wandb API 키를 찾을 수 없어, wandb 연동이 비활성화되었습니다.")


## 5-2. SFTConfig 생성
sft_config = SFTConfig(
    # --- SFTTrainer 특화 인자 ---
    dataset_text_field="text",
    max_seq_length=2048,

    # --- TrainingArguments 인자 ---
    output_dir="Qwen2.5-14B-KB-Finance-LoRA", # 훈련 결과물이 저장될 폴더
    report_to=report_to,                      # 환경에 따라 설정된 값 (wandb 또는 none)
    run_name="qwen2.5-14b-lora-ep2",           # wandb에 표시될 실행 이름

    num_train_epochs=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,

    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,

    bf16=True,
    optim="paged_adamw_8bit",

    logging_strategy="steps",
    logging_steps=25,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
)

print("\n--- SFTConfig 준비 완료 ---")

RunPod 환경 변수를 찾을 수 없어 Colab userdata로 wandb 키를 확인합니다.
✅ wandb 연동이 활성화되었습니다.

--- SFTConfig 준비 완료 ---


# [6단계] 트레이너 생성 (DataCollator 적용)

## 6-1. 답변만 학습을 위한 DataCollator 생성 (멘토님 조언 4.2)

In [35]:
# 이 DataCollator는 모델에게 'assistant'의 답변 부분만 학습하도록 지시한다.
# instruction_template은 질문의 시작 부분을, response_template은 답변의 시작 부분을 알려주는 역할.
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    instruction_template="<|im_start|>user",
    response_template="<|im_start|>assistant",
)

## 6-2. SFTTrainer 생성

In [36]:
# 이제 모든 준비물을 합쳐서 최종 트레이너를 만든다.
trainer = SFTTrainer(
    model=lora_model,          # LoRA가 적용된 모델
    tokenizer=tokenizer,       # 패딩 토큰이 설정된 토크나이저
    args=sft_config,           # SFTConfig로 만든 훈련 계획
    train_dataset=final_dataset["train"], # 훈련용 데이터셋
    eval_dataset=final_dataset["test"],   # 평가용 데이터셋 (로그에 평가 loss를 남기려면 필요)
    data_collator=data_collator, # 답변만 학습시키는 데이터 콜레이터
)

print("\n--- SFTTrainer 준비 완료 ---")
print("이제 trainer.train()을 호출하여 훈련을 시작할 수 있습니다.")

Map:   0%|          | 0/30606 [00:00<?, ? examples/s]

Map:   0%|          | 0/1708 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 136.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 21201 has 14.73 GiB memory in use. Of the allocated memory 14.61 GiB is allocated by PyTorch, and 5.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# [7단계] 훈련 시작

In [ ]:
print("\n--- 파인튜닝을 시작합니다 ---")
trainer.train()
print("\n--- 파인튜닝 완료 ---")

# [8단계] 모델 최종 저장 및 업로드

In [ ]:
# push_to_hub=True 옵션 덕분에 trainer.train()이 끝나면 자동으로 업로드된다.
print(f"\n✅ 최종 모델이 Hugging Face Hub에 업로드되었습니다.")
print(f"URL: https://huggingface.co/{final_model_repo_id}")